# **🧠 健康知識與症狀導引系統**

本系統結合 **自行建構的 RAG（Retrieval-Augmented Generation）資料查詢架構** 與 **2-Stage CoT 機制**，提供一個簡單直覺的互動介面，協助使用者釐清健康症狀並取得可靠資訊。

🔍 系統特色：
- ✅ 根據你輸入的**症狀、感受或描述**，從醫學知識資料庫中擷取相關說明（包含 MedlinePlus、Mayo Clinic、台灣衛福部等）。
- ✅ 可選擇啟用 **推理模式（2-Stage CoT）**，透過模型多步思考進行更深入的健康分析與建議。
- ✅ 支援條列式回答需求，讓你更清楚掌握資訊要點。

<hr>

---

### **模型與專案目標說明**

### 一、使用模型介紹

本系統主要使用以下模型與架構：

- **向量化模型**：`intfloat/multilingual-e5-small`  
  一款多語言語意嵌入模型，適合醫療知識文件與使用者問題的語意比對。
  
- **語言生成模型**：`Groq/llama3-70b-8192`  
  採用 Groq 平台提供的 LLaMA 3 大型語言模型，具備快速且邏輯清晰的生成能力，支援中文對話。
  
- **向量資料庫與檢索**：`LangChain + FAISS`  
  搭配 FAISS 向量資料庫管理自建醫療文本，並透過檢索強化生成（RAG）架構提升回答準確性。

---

### 二、專案目標

本專案目標在於：

- **整合公開醫療知識來源（如 MedlinePlus）建立向量資料庫**
- **結合 RAG 與 CoT 技術打造中文健康問答系統**
- **提供溫暖、清楚、具參考性的健康導引建議**
- **實作完整互動流程（UI + 檢索 + 推理）並可部署於 Colab**

---
---

### **1. 準備 RAG 系統**


📦 本系統使用自行建立的向量資料庫（`faiss_db_final.zip`），  
資料內容來自健康相關網站，包含 MedlinePlus、Mayo Clinic 及台灣衛福部公開資料。

若您在本地環境使用，請先執行以下步驟載入資料庫：

In [2]:
!pip install gdown

import gdown

file_id = "1n3tG5Z_aZl7JtbP5NAqGIls4kn0v8SwM"

# 用gdown下載，檔名叫 faiss_db.zip
gdown.download(f"https://drive.google.com/uc?id={file_id}", "faiss_db_final.zip", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1n3tG5Z_aZl7JtbP5NAqGIls4kn0v8SwM
To: /content/faiss_db_final.zip
100%|██████████| 18.4M/18.4M [00:00<00:00, 37.5MB/s]


'faiss_db_final.zip'

#### **1.1 此處檢查輸入雲端連結之大小**

In [3]:
!ls -lh faiss_db_final.zip

-rw-r--r-- 1 root root 18M May 30 11:34 faiss_db_final.zip


#### **1.2 解壓縮本地下載之 zip 檔**

In [4]:
!unzip faiss_db_final.zip

Archive:  faiss_db_final.zip
   creating: faiss_db/
  inflating: faiss_db/index.faiss    
  inflating: faiss_db/index.pkl      


#### **1.3 安裝並引入必要套件**
本系統需搭配以下 Python 套件運行，包括 RAG（檢索增強生成）、向量資料庫與介面建構等功能。

請於執行前確保安裝以下套件（建議於 Colab 或本地虛擬環境執行）：

In [5]:
!pip install -U langchain langchain-community sentence-transformers faiss-cpu gradio openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.5/720.5 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 119.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/8

#### **1.4 載入核心模組與套件**

本段程式碼匯入系統運作所需的主要元件，包括向量資料庫、語意嵌入模型、聊天模型與問答鏈設定：


In [6]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

In [7]:
from openai import OpenAI
import gradio as gr

#### **1.5 客製化語意嵌入：E5 模型格式調整**

E5 系列模型（如 `intfloat/e5-base-v2`）在訓練時採用了格式化提示（prompt formatting），需要對輸入文字加入前綴才能正確取得語意嵌入效果。

以下我們透過繼承 `HuggingFaceEmbeddings` 類別，建立自定義的 `CustomE5Embedding` 類別，確保在向量化文件與查詢時，自動套用必要的前綴詞格式：


In [8]:
class CustomE5Embedding(HuggingFaceEmbeddings):
    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

#### **1.6 建立檢索系統：載入語意嵌入模型與 FAISS 向量資料庫**

我們使用自定義的 `CustomE5Embedding` 搭配 multilingual E5 模型，將查詢與知識資料轉換成向量，供 FAISS 向量資料庫進行高效率相似度比對。


In [9]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/498k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

<hr>

### **2. 準備 CoT 系統**


#### **2.1 安裝 aisuite：整合 AI 模型與提示工程工具**

透過以下指令安裝 `aisuite` 套件與其所有可用擴充功能模組，方便進行大語言模型推論、CoT 推理模板設計與多模型 API 整合。


In [10]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 68.4 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.16.1 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


#### **2.2 串接 Groq API 模型，進行自訂化語言生成**

本段程式碼用於串接 Groq 平台上的大型語言模型（如 LLaMA3），並透過自定 `reply` 函數進行自然語言回應生成。


In [11]:
import os
import re
import gradio as gr
import aisuite as ai
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.chains import RetrievalQA

# 讀取 API Key
from google.colab import userdata
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY'] = api_key

####  `reply()` 函式說明

這個函式是我們用來呼叫 Groq 語言模型（如 LLaMA3）的自訂介面，具備以下特點：

- `prompt`：使用者輸入的主要問題
- `system`：設定回答風格（例如：請用台灣常用中文回答）
- `model`：選擇使用的模型，預設為 `"llama3-70b-8192"`

此函式特別適用於：
- ✅ RAG 模式（檢索增強生成）
- ✅ 2-Stage CoT 推理（分兩步生成答案）
- ✅ 自訂回答語氣（條列式、一步步推理等）

另外也特別透過 `SyncHttpxClientWrapper()` 建立乾淨的連線，避免 Colab 或內部網路環境的 proxy 汙染導致錯誤。


In [30]:
from groq import Groq
from groq._base_client import SyncHttpxClientWrapper

def reply(prompt, system="請用台灣習慣的中文回覆。", model="llama3-70b-8192"):
    # 建立乾淨的 httpx client，不帶 proxies
    client = Groq(
        api_key=os.environ["GROQ_API_KEY"],
        http_client=SyncHttpxClientWrapper()  # 覆蓋可能預設有 proxies 的設定
    )

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(model=model, messages=messages)
    return response.choices[0].message.content

<hr>

### **3. 模型設計與準備**

#### **3.1 Prompt 設計說明**

本系統設計多組提示語（prompts），用以支援不同回應模式（聊天、摘要、推理），搭配健康資料庫提升回答品質。


#### 📌 `system_prompt`
- 角色：溫柔可靠的健康導引助手  
- 能力：結合檢索與推理，輔助理解症狀  
- 限制：不診斷、不取代醫療  
- 語氣：自然親切、鼓勵補充資訊  


#### 💬 `prompt_template_chat`
- 適用自然對話，親切互動
- 鼓勵補充症狀背景
- 避免條列，口語化說明


#### 📋 `prompt_template_bullet`
- 條列方式整理重點
- 提供清楚建議與保健方向
- 可融入可信資料來源

#### 🧾 `prompt_template`
- 協助分析症狀與健康主題
- 提出具體保健建議
- 引用官方資訊來源（如 Mayo Clinic）


#### 🧠 `prompt_template_cot`（CoT 模式）
- 進行邏輯推理分析
- 從描述中歸納可能議題與成因
- 最後提醒可尋求專業協助


這些 prompt 讓系統能因應不同需求提供溫暖、科學且實用的健康說明。

In [38]:
system_prompt = """
請使用台灣常見的中文與說話方式回應！

你是一位溫柔可靠的健康症狀導引助手，擁有醫學資訊檢索與邏輯推理的能力。當使用者描述身體不適、症狀或生活狀況時，你會結合 MedlinePlus、Mayo Clinic、台灣衛福部等資料，進行合適的推理與健康建議說明。

你的目標是幫助使用者更了解自身狀況，並協助他們採取初步保健行動，但不進行醫療診斷或判定病名。

一開始對話時，你會用自然、親切的語氣，像一位可靠的諮詢者。若使用者提供兩項以上的具體資訊（例如症狀加上出現時間、或伴隨的生活情境），你可以進一步推論可能的健康狀況與背景機轉，提供資訊與建議。

如果資訊不夠清楚，你會以鼓勵或提問的方式，引導對方補充說明。過程中避免使用絕對語氣（如「一定」或「絕對」），並可提醒使用者：如果症狀持續或惡化，應尋求專業醫療協助。
"""

prompt_template_chat = """
以下是資料庫中與提問相關的資訊，請參考內容後，用自然親切的方式回應使用者，並協助釐清目前的身體狀況或症狀特性。

{retrieved_chunks}

使用者提問：「{question}」

請注意：
- 若提問內容仍偏籠統或只提及一項感受，請試著溫和鼓勵對方補充更多資訊，例如出現多久了、什麼時候會發作、是否影響生活等。
- 回覆時請避免使用條列式，採用口語化且引導性的語氣進行互動。
- 如感覺與營養或生活習慣可能有關，也可以提出初步建議（例如飲食或作息調整），幫助使用者做簡單判斷。

請以真誠、科學且關懷的方式進行對話。
"""


prompt_template_bullet = """
以下是系統擷取的資訊，請協助使用者整理出重點內容，並提供清楚的建議與說明：

{retrieved_chunks}

使用者輸入：「{question}」

請試著幫助使用者了解：
可能與此狀況相關的健康主題是什麼？有哪些常見成因或風險可能值得留意？在日常生活中，有哪些實用的保健建議（例如營養補充、作息調整）？

若能從資料中找到出處（如 MedlinePlus、Mayo Clinic 或台灣衛福部），也請自然地融入說明中。

如果資料無法明確對應此症狀，請坦率說明，並提醒使用者考慮尋求專業醫療協助。
"""

prompt_template = """
請根據以下擷取的資料，協助使用者進行初步的健康狀況分析，並提供有幫助的建議：

{retrieved_chunks}

使用者輸入：「{question}」

回覆時可引導使用者理解：
目前的狀況可能與哪些健康主題有關？這些主題背後有什麼典型的成因、風險因素或機轉？是否可以從生活保健、飲食補充、作息調整等角度提出具體建議？

如果資料中出現來自官方可信來源（如 Mayo Clinic 或台灣衛福部）的內容，也請在說明中一併引用。

請用清楚、有同理心的語氣，幫助使用者對自身狀況有更全面的理解。
"""


prompt_template_cot = """
以下是使用者描述的症狀與背景資訊：「{question}」

請參考下方資料，運用多步驟邏輯思考，協助推理出可能的健康議題與背後原因：

{retrieved_chunks}

你可以先從使用者的描述中抓出觀察重點，例如症狀的出現順序、時間點、是否與生活背景有關。接著，再根據資料內容與這些觀察，思考有哪些可能的健康主題、潛在成因，並提出建議的理解方向與保健方式。

請避免直接判斷病名，而是以「可能與……有關」、「可考慮……因素」等語氣呈現。最後可補充提醒：「若情況持續或惡化，建議尋求醫療協助以確認狀況。」
"""


#### **3.2 回應邏輯：`chat_with_rag`**

此函數整合聊天紀錄、使用者輸入與檢索內容，搭配不同提示語（prompt）生成適當回應。


#### 🔁 主要參數
- `user_input`：使用者輸入內容
- `chat_history_local`：近期對話紀錄（取最後 5 則）
- `bullet_points`：是否啟用條列模式
- `use_cot`：是否啟用 2-Stage Chain-of-Thought 模式


#### 📌 回應模式邏輯
1. 根據輸入選擇對應的 prompt（一般/條列/推理）
2. 使用 retriever 擷取相關知識內容
3. 將資料與使用者輸入組合，產生最終 prompt


#### 🧠 CoT 推理模式（`use_cot=True`）
- **Stage 1**：先要求模型進行逐步邏輯推理
- **Stage 2**：再根據推理結果統整建議與可能判斷


#### 💬 一般與條列模式
- 使用 `prompt_template` 或 `prompt_template_bullet` 組合輸入與擷取內容
- 呼叫 `reply()` 取得模型回應


最後回傳回應內容與更新後的對話歷程（`chat_history_local`）。


In [45]:
chat_history = []
model = "llama3-70b-8192"

def chat_with_rag(user_input, chat_history_local, bullet_points=False, use_cot=False):
    past_messages = [m[0] for m in chat_history_local[-5:]]
    combined_context = " ".join(past_messages + [user_input])

    if use_cot:
        prompt_to_use = prompt_template_cot
    elif bullet_points:
        prompt_to_use = prompt_template_bullet
    else:
        prompt_to_use = prompt_template

    docs = retriever.invoke(user_input)
    retrieved_chunks = "\n\n".join([doc.page_content for doc in docs])

    if use_cot:
        stage1_prompt = f"根據以下醫療資訊，請你一步一步推理、列出與「{user_input}」相關的可能症狀與成因：\n\n{retrieved_chunks}\n\n請寫出詳細推理步驟，不要直接給結論。"
        stage1_response = reply(prompt=stage1_prompt, model=model)
        stage2_prompt = f"{stage1_response}\n\n根據以上推理，請統整出你對此症狀問題的建議與可能判斷。"
        final_response = reply(prompt=stage2_prompt, model=model)
    else:
        full_prompt = prompt_to_use.format(question=user_input, retrieved_chunks=retrieved_chunks)
        final_response = reply(prompt=full_prompt, model=model)

    chat_history_local.append((user_input, final_response))
    return final_response, chat_history_local


<hr>

### **4. 建立使用者介面**

### **4.1 Gradio 前端介面設計：`demo`**

本段程式碼建立互動式聊天介面，整合 RAG 模型與健康導引功能，提供使用者輸入與回應對話的操作平台。

#### 💬 主要組件
- `gr.Markdown`：說明系統用途與使用示例
- `gr.Chatbot()`：顯示雙向對話內容
- `gr.Textbox()`：使用者輸入症狀
- `gr.Checkbox()`：啟用 2-Stage CoT 推理模式選項

#### 🔁 回應邏輯 `respond()`
- 自動偵測輸入文字是否含有「列點」、「整理」等字眼 → 啟用條列模式
- 呼叫 `chat_with_rag()` 處理對話、檢索與生成
- 更新對話歷程並顯示回應內容


In [68]:
import gradio as gr
import re

css = """
body, .gradio-container {
    background-color: #e6f0fa !important;
    font-family: "Helvetica Neue", sans-serif;
}

/* Chatbot 外觀 */
.gr-chatbot, .chatbot {
    background-color: #cce0f5 !important;
    border-radius: 10px;
    border: 1px solid #99c2e6;
    padding: 12px;
}

/* 對話訊息氣泡 */
.message, .message.user, .message.bot {
    border-radius: 8px;
    padding: 10px 14px;
    margin: 6px 0;
}
.message.user {
    background-color: #b3d9ff !important;
    color: #003366 !important;
}
.message.bot {
    background-color: #d9eaff !important;
    color: #002244 !important;
}

/* 輸入框、checkbox、按鈕 */
textarea, input[type="text"], .gr-textbox, .gr-checkbox, .svelte-1ipelgc {
    background-color: #ffffff !important;
    border: 1px solid #99c2e6 !important;
    border-radius: 6px;
    padding: 10px;
}

/* 提交按鈕樣式 */
button, .gr-button, .gr-button-primary {
    background-color: #6699cc !important;
    color: white !important;
    border: none !important;
    border-radius: 6px !important;
    padding: 10px 16px !important;
}
button:hover {
    background-color: #4d88cc !important;
}

/* Checkbox 樣式 */
input[type="checkbox"] {
    accent-color: #6699cc;
}

/* Markdown 標題與說明區 */
.gr-markdown {
    background-color: #e0ecff;
    padding: 12px;
    border-radius: 6px;
    color: #003366;
    font-size: 1.2em;
}
"""

with gr.Blocks(css=css) as demo:
    gr.Markdown("""
    # 🩺 健康症狀導引系統

    請輸入你目前的症狀、身體不適或主觀感受，系統會根據資料庫資訊提供說明與建議。

    範例輸入：
    - 最近喉嚨痛，有點咳嗽。
    - 一直覺得很疲倦，該補什麼？

    ❗提醒：本系統僅提供參考，無法取代專業診斷。
    """)

    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="請輸入你的症狀、身體不適或感受...")
    use_cot = gr.Checkbox(label="使用推理模式（2-Stage CoT）", value=False)

    def respond(message, chat_history_local, cot_flag):
        request_bullets = bool(re.search(r"(列點|條列|幾點|整理)", message))
        response, updated_history = chat_with_rag(
            user_input=message,
            chat_history_local=chat_history_local,
            bullet_points=request_bullets,
            use_cot=cot_flag
        )
        return "", updated_history

    msg.submit(respond, [msg, chatbot, use_cot], [msg, chatbot])

demo.launch(debug=True)


<ipython-input-68-9d80a0d86ad3>:81: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c8b4113f66098ac810.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7864 <> https://c8b4113f66098ac810.gradio.live
